In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial.distance import cdist
import numpy as np
from scipy.sparse import lil_matrix

csv_path = r"C:\Users\a0494\Desktop\New folder (3)\fire_archive_M-C61_388698.csv"

# Зчитайте дані з CSV-файлу
df = pd.read_csv(csv_path)

df.head()


# Додайте новий стовпець "id"
df['id'] = range(1, len(df) + 1)
df['group_id'] = -1

df_short = df[['latitude', 'longitude', 'acq_date', 'id', 'group_id']]
df_short.head()

from datetime import datetime, timedelta

def date_range(start_date, end_date, format='%Y-%m-%d'):
    current_date = datetime.strptime(start_date, format)
    end_date = datetime.strptime(end_date, format)

    while current_date <= end_date:
        yield current_date.strftime(format)
        current_date += timedelta(days=1)

# Використання функції для створення циклу дат
start_date = '2020-01-01'
end_date = '2021-12-12'
date_format = '%Y-%m-%d'


In [ ]:
def replace_group_id(df_short_classificated, k, new_value):
    # Замінити всі значення group_id, рівні k, на new_value
    df_short_classificated.loc[df_short_classificated['group_id'] == k, 'group_id'] = new_value


def clasterisation(filtered_rows, filtered_rows_pre, k, boolean_id_matrix):
    # Об'єднати filtered_dates і filtered_rows_pre
    df_combined = pd.concat([filtered_rows, filtered_rows_pre], ignore_index=True)

    # Розрахунок відстаней між точками
    distances = cdist(df_combined[['latitude', 'longitude']], df_combined[['latitude', 'longitude']], metric='euclidean')

    # Задайте поріг для відстані
    distance_threshold = 1.5  # 2 км

    # Знайдіть пари точок, які задовольняють умову відстані
    pairs = np.where(distances <= distance_threshold)

    # Проініціалізуйте group_id у filtered_dates значеннями з filtered_rows_pre
    #filtered_rows['group_id'] = filtered_rows.apply(lambda row: row['group_id'] if not pd.isna(row['group_id']) else np.nan, axis=1)
    for i, j in zip(*pairs):
        if i!=j:
            id_i_r = df_combined.at[i, 'id']
            id_j_r = df_combined.at[j, 'id']
            #print(id_i_r, id_j_r)
            id_i = df_combined.at[i, 'group_id']
            id_j = df_combined.at[j, 'group_id']
            has_id_i = id_i != -1
            has_id_j = id_j != -1
            if has_id_i and has_id_j:
                replace_group_id(df_short_classificated, id_i, id_j)
            elif has_id_i:
                # Якщо точка i має group_id, присвойте його точці j
                df_combined.loc[df_combined['id'] == id_i_r, 'group_id'] = id_i
            elif has_id_j:
                df_combined.loc[df_combined['id'] == id_j_r, 'group_id'] = id_j
            else:
                 k += 1
                 df_combined.loc[df_combined['id'] == id_i_r, 'group_id'] = k
                 df_combined.loc[df_combined['id'] == id_j_r, 'group_id'] = k
    # Перевірка, чи всі точки мають id
    #if 'group_id' not in filtered_dates.columns:
    #    # Якщо 'id' відсутній у стовпцях, додайте його
    #    filtered_dates['group_id'] = range(1, len(filtered_dates) + 1)
    #print(df_combined, filtered_rows)
    df_combined_cropped = df_combined[df_combined['id'].isin(filtered_rows['id'])]
    #print(len(df_combined_cropped),"::", len(df_combined))
   # print(filtered_dates)
    return df_combined_cropped, k, boolean_id_matrix
# Розмірність матриці
matrix_size = 37000

# Створити матрицю з False (булевий тип за замовчуванням)
boolean_id_matrix = np.full((matrix_size, matrix_size), False, dtype=bool)



columns = ['latitude', 'longitude', 'acq_date', 'id', 'group_id']
df_short_classificated = pd.DataFrame(columns=columns)
filtered_rows_pre = df_short_classificated
filtered_rows = filtered_rows_pre
k = 0
n=0
import time
import warnings
warnings.filterwarnings('ignore')

for date_str in date_range(start_date, end_date, date_format):
    n+=1
    if(n>180):
        start_time = time.time()  # Запускаємо таймер на початку циклу
        df_short_classificated = df_short_classificated.append(filtered_rows_pre, ignore_index=True)
        filtered_rows = df_short[df_short['acq_date'] == date_str]
        filtered_rows_pre, k, boolean_id_matrix = clasterisation(filtered_rows, filtered_rows_pre, k, boolean_id_matrix)
        end_time = time.time()  # Зупиняємо таймер після завершення обчислень
        print(f"{date_str} :: {k} :: Time: {end_time - start_time} seconds")








In [ ]:
df_short_classificated = df_short_classificated.sort_values(by='group_id')

# Вивести результат
print(df_short_classificated.tail(20))

In [ ]:
print(df_short_classificated.tail(20))

In [ ]:
df_short_classificated.to_csv(r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\test.csv", index=False)

In [ ]:
import pandas as pd

# Припускаючи, що у вас є стовпець 'id' який є унікальним ключем
# і у df є стовпець 'group_id', який вам потрібно оновити
df_final = pd.merge(df, df_short_classificated[['id', 'group_id']], on='id', how='left', suffixes=('', '_new'))

# Оновлення значень group_id за умовою
df_final['group_id'] = df_final.apply(lambda row: row['group_id_new'] if row['group_id_new'] != -1 else row['group_id'], axis=1)

# Видалення допоміжного стовпця
df_final = df_final.drop(columns=['group_id_new'])

In [ ]:
df_final = df_final.sort_values(by='group_id')
df_final.to_csv(r"C:\Users\a0494\Desktop\New folder (3)\Кластерізовані\final.csv", index=False)